In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-02-25 03:51:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-02-25 03:51:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-02-25 03:51:57--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [2]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
import pandas as pd
import json
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import requests

In [4]:
def get_raw_data():
    train_data_url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
    test_data_url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/dev.json"
    train_data_json = requests.get(train_data_url).json()
    test_data_json = requests.get(test_data_url).json()
    return train_data_json, test_data_json

CLASSES = ['PREAMBLE', 'NONE', 'FAC', 'ARG_RESPONDENT', 'RLC', 'ARG_PETITIONER', 'ANALYSIS', 'PRE_RELIED', 'RATIO', 'RPC', 'ISSUE', 'STA', 'PRE_NOT_RELIED']
train_data_json, test_data_json = get_raw_data()

In [5]:
df_train = pd.DataFrame(columns = ["text", "output"])
p = 0
for i in tqdm(range(len(train_data_json))):
  for j in range(len(train_data_json[i]["annotations"])):
     for k in range(len(train_data_json[i]["annotations"][j]["result"])):
       text = train_data_json[i]["annotations"][j]["result"][k]["value"]["text"]
       output = train_data_json[i]["annotations"][j]["result"][k]["value"]["labels"][0]
       df_train.loc[p] = [text, output]
       p+=1

100%|██████████| 247/247 [01:11<00:00,  3.44it/s]


In [6]:
df_train.head()

,text,output
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE
1,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE
2,This Criminal Appeal is filed under Section 37...,PREAMBLE
3,\n\n This appeal coming on for hearing t...,PREAMBLE
4,\n Heard the learned Counsel for the app...,NONE


In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_train["text"].tolist())
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(df_train["text"].tolist())

In [8]:
padded_docs = tf.keras.utils.pad_sequences(encoded_docs, maxlen=380, padding='post')

In [9]:
label = list(set(df_train["output"].tolist()))

In [10]:
index_map = {}
for i in range(len(label)):
  index_map[label[i]] = i

df_train["label"] = df_train["output"].map(index_map)

In [11]:
df_train.head()

,text,output,label
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE,5
1,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE,5
2,This Criminal Appeal is filed under Section 37...,PREAMBLE,5
3,\n\n This appeal coming on for hearing t...,PREAMBLE,5
4,\n Heard the learned Counsel for the app...,NONE,1


In [12]:
final_label = df_train["label"].tolist()

In [13]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	all_values = line.split()
	curr_word = all_values[0]
	coefficients = np.asarray(all_values[1:], dtype='float32')
	embeddings_index[curr_word] = coefficients
f.close()

In [14]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [15]:
final_labels = np.array(final_label)

In [16]:
num_classes = len(index_map)
final_labels = tf.keras.utils.to_categorical(final_labels, num_classes=num_classes)

x_test, x_train = padded_docs[:3000], padded_docs[3000:]
y_test, y_train = final_labels[:3000], final_labels[3000:]

In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=380, trainable=False))
model.add(tf.keras.layers.Conv1D(32, 3, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 380, 100)          2192000   
                                                                 
 conv1d (Conv1D)             (None, 378, 32)           9632      
                                                                 
 flatten (Flatten)           (None, 12096)             0         
                                                                 
 dense (Dense)               (None, 13)                157261    
                                                                 
Total params: 2358893 (9.00 MB)
Trainable params: 166893 (651.93 KB)
Non-trainable params: 2192000 (8.36 MB)
_________________________________________________________________
None


In [19]:
model.fit(x_train, y_train, epochs=10, verbose=2)

Epoch 1/10
813/813 - 38s - loss: 1.6367 - accuracy: 0.4820 - 38s/epoch - 46ms/step
Epoch 2/10
813/813 - 22s - loss: 1.3291 - accuracy: 0.5687 - 22s/epoch - 28ms/step
Epoch 3/10
813/813 - 24s - loss: 1.1777 - accuracy: 0.6158 - 24s/epoch - 29ms/step
Epoch 4/10
813/813 - 25s - loss: 1.0775 - accuracy: 0.6430 - 25s/epoch - 31ms/step
Epoch 5/10
813/813 - 27s - loss: 1.0035 - accuracy: 0.6682 - 27s/epoch - 33ms/step
Epoch 6/10
813/813 - 26s - loss: 0.9449 - accuracy: 0.6842 - 26s/epoch - 32ms/step
Epoch 7/10
813/813 - 25s - loss: 0.8968 - accuracy: 0.6994 - 25s/epoch - 31ms/step
Epoch 8/10
813/813 - 23s - loss: 0.8544 - accuracy: 0.7125 - 23s/epoch - 28ms/step
Epoch 9/10
813/813 - 23s - loss: 0.8167 - accuracy: 0.7245 - 23s/epoch - 28ms/step
Epoch 10/10
813/813 - 23s - loss: 0.7877 - accuracy: 0.7337 - 23s/epoch - 29ms/step


In [20]:
y_pred = model.predict(x_test, verbose=2)

94/94 - 1s - 1s/epoch - 15ms/step


In [21]:
Y_pred = np.argmax(y_pred,axis=1)
Y_test = np.argmax(y_test,axis=1)

In [22]:
from sklearn.metrics import f1_score
score=f1_score(Y_test, Y_pred, average='macro')
print("F1Score:", score)

F1Score: 0.3116935596451801


In [23]:
score

0.3116935596451801